In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, Dropout
from keras.models import Sequential

from sklearn.decomposition import PCA

from sklearn.neural_network import MLPRegressor

import numpy as np

import xgboost as xgb

In [2]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [3]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [5]:
df["concatenated"] = df['Question'] + " " + df['Student Answer']

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["concatenated"]], df["Grade"], test_size=0.1, random_state=42)

In [7]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [8]:
train_X["concatenated"] = train_X["concatenated"].apply(extract_pos).apply(process_dataset)

test_X["concatenated"] = test_X["concatenated"].apply(extract_pos).apply(process_dataset)

In [9]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 2)
(2197, 2)


In [64]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]

In [65]:
sentences = list(train_X["concatenated"])
sentences = [i.split() for i in sentences]

In [118]:
vocab = []
for words_list in sentences:
  vocab.extend(words_list)


In [66]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [67]:
model = Word2Vec(sentences, min_count=1)

In [121]:
def get_wordvec_embeddings(data, default_embedding=np.zeros(100)):

  words = data.split()
  embedding = []
  for word in words:
      if word in vocab:
          embedding.append(model.wv[word])
      else:
          embedding.append(default_embedding)

  return np.mean(np.array(embedding),axis = 0)

In [122]:
train_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":train_X["concatenated"]})
test_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":test_X["concatenated"]})

In [123]:
train_Embeddings_df["Q_Student_Answer_Embeddings"] = train_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_wordvec_embeddings)

test_Embeddings_df["Q_Student_Answer_Embeddings"] = test_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_wordvec_embeddings)

In [126]:
train_df = pd.DataFrame(list(train_Embeddings_df["Q_Student_Answer_Embeddings"]))
test_df = pd.DataFrame(list(test_Embeddings_df["Q_Student_Answer_Embeddings"]))

In [127]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.090506,0.184706,0.075666,0.026179,0.147399,-0.445735,0.107368,0.612309,-0.158350,-0.133800,...,0.407920,0.168075,-0.039234,0.080622,0.519608,0.305581,0.148112,-0.240994,0.136609,-0.096575
1,-0.178085,0.255821,0.152500,0.047127,0.200049,-0.637159,0.146649,0.839343,-0.184886,-0.181625,...,0.519592,0.189374,-0.055452,0.115243,0.702580,0.417600,0.171190,-0.302176,0.184134,-0.147428
2,-0.113006,0.212421,0.063330,0.041309,0.151697,-0.481584,0.076797,0.646748,-0.122861,-0.135109,...,0.398147,0.166007,-0.066887,0.059343,0.523129,0.313610,0.143620,-0.232231,0.130939,-0.102449
3,-0.113879,0.226316,0.082405,0.028110,0.177055,-0.502064,0.096778,0.674035,-0.142565,-0.141574,...,0.425510,0.178514,-0.069910,0.065443,0.539205,0.331014,0.173127,-0.240637,0.139536,-0.100930
4,-0.110679,0.186999,0.066150,0.037564,0.135441,-0.415815,0.068074,0.570278,-0.109102,-0.115339,...,0.351447,0.155064,-0.059108,0.053890,0.467535,0.278454,0.119369,-0.206029,0.124549,-0.085475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,-0.125194,0.276603,0.048344,0.047084,0.219304,-0.544069,0.105629,0.739338,-0.198107,-0.137244,...,0.438747,0.278034,-0.077180,0.035942,0.627906,0.356671,0.227511,-0.266871,0.190081,-0.095751
2193,-0.148883,0.260394,0.116377,0.024307,0.215250,-0.559174,0.084303,0.784445,-0.154858,-0.160219,...,0.469402,0.213584,-0.109269,0.084580,0.602026,0.372968,0.205244,-0.277361,0.170467,-0.133185
2194,-0.128669,0.264821,0.070469,0.047707,0.226657,-0.626120,0.100708,0.815830,-0.216452,-0.162019,...,0.522847,0.265900,-0.095708,0.081721,0.703074,0.375545,0.259051,-0.329146,0.215779,-0.160806
2195,-0.067546,0.214511,0.033717,0.031647,0.202887,-0.483220,0.096772,0.646043,-0.205852,-0.114938,...,0.431720,0.247153,-0.048876,0.064055,0.598616,0.305046,0.229022,-0.298785,0.182692,-0.118746


In [128]:
pca = PCA(n_components=100)  # Specify the number of components you want to keep
train_df_pca = pca.fit_transform(train_df)

In [129]:
test_df_pca = pca.transform(test_df)

In [130]:
ncols = train_df_pca.shape[1]

In [131]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(ncols,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_df_pca, train_y, epochs=10, batch_size=32, verbose=1)

y_true = test_y
y_preds = model.predict(test_df_pca)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
cnn_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
cnn_mse = mean_squared_error(y_true, np.round(y_preds, 1))
cnn_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
cnn_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
cnn_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(cnn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(cnn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(cnn_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(cnn_mape, 2))
print("R2 Score: ", np.round(cnn_r2, 2))

Epoch 1/10
69/69 [==============================] - 10s 4ms/step - loss: 5.5420
Epoch 2/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6345
Epoch 3/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6063
Epoch 4/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6293
Epoch 5/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6122
Epoch 6/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5901
Epoch 7/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6139
Epoch 8/10
69/69 [==============================] - 0s 3ms/step - loss: 2.5940
Epoch 9/10
69/69 [==============================] - 0s 3ms/step - loss: 2.6099
Epoch 10/10
8/8 [==============================] - 0s 2ms/step
Mean Absolute Error (MAE):  1.16
Mean Squared Error (MSE):  1.95
Root Mean Squared Error (RMSE):  1.39
Mean Absolute Percentage Error (MAPE):  623913416057267.1
R2 Score:  0.02


In [132]:
# Create an MLP regressor with a single linear output layer
mlp = MLPRegressor(hidden_layer_sizes=(100,50,25,1), activation='identity', solver='lbfgs')

# Train the MLP regressor
mlp.fit(train_df_pca, train_y)

y_preds = mlp.predict(test_df_pca)

y_true = test_y

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
mlp_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
mlp_mse = mean_squared_error(y_true, np.round(y_preds, 1))
mlp_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
mlp_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
mlp_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(mlp_mae, 2))
print("Mean Squared Error (MSE): ", np.round(mlp_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(mlp_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(mlp_mape, 2))
print("R2 Score: ", np.round(mlp_r2, 2))

Mean Absolute Error (MAE):  1.11
Mean Squared Error (MSE):  1.78
Root Mean Squared Error (RMSE):  1.34
Mean Absolute Percentage Error (MAPE):  545584170051801.7
R2 Score:  0.1


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [133]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df_pca, train_y)

preds = LR_model.predict(test_df_pca)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.11
Mean Squared Error (MSE):  1.82
Root Mean Squared Error (RMSE):  1.35
Mean Absolute Percentage Error (MAPE):  531240935636764.9
R2 Score:  0.09


In [134]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df_pca, train_y)  #fit the model
    pred=knn_model.predict(test_df_pca) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

y_true = test_y
knn_preds = knn_model.predict(test_df_pca)

min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.23
Mean Squared Error (MSE):  2.8
Root Mean Squared Error (RMSE):  1.67
R2 Score:  -0.39


In [135]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.31
Mean Squared Error (MSE):  2.23
Root Mean Squared Error (RMSE):  1.49
Mean Absolute Percentage Error (MAPE):  479771225609673.6
R2 Score:  -0.13


In [136]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df_pca, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df_pca))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.1
Mean Squared Error (MSE):  1.85
Root Mean Squared Error (RMSE):  1.36
Mean Absolute Percentage Error (MAPE):  492638653116446.4
R2 Score:  0.07


In [137]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df_pca, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df_pca))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.14
Mean Squared Error (MSE):  2.17
Root Mean Squared Error (RMSE):  1.47
Mean Absolute Percentage Error (MAPE):  466903789584183.6
R2 Score:  -0.09
